In [1]:
url= "https://www.exeter.ac.uk/study/studyinformation/modules/?prog=classics&year=2025/6"
filepath=r"C:\Users\Praduman\Downloads\chrt.csv"  #save in your local directory


In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# Set up headless Chrome
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# Initialize WebDriver
driver = webdriver.Chrome(options=options)

# URL of the modules page
url = url
driver.get(url)

# Allow the page to load (increase sleep time if necessary)
time.sleep(5)

# Extract all anchor tags on the page
anchor_tags = driver.find_elements(By.TAG_NAME, "a")

# Filter the links that seem relevant to the module descriptors (e.g., those containing "module")
module_links = [tag.get_attribute("href") for tag in anchor_tags if "module" in tag.get_attribute("href")]
c=[]
# Check if we got any module-related links
if module_links:
    print("Found module links:")
    for link in module_links:
        c.append(link)
        print(link)
else:
    print("No relevant module links found.")

# Close the browser
driver.quit()

Found module links:
https://www.exeter.ac.uk/study/studyinformation/modules/#main-col
https://www.exeter.ac.uk/study/studyinformation/modules/#
https://www.exeter.ac.uk/study/studyinformation/modules/#site-search-collapse
https://www.exeter.ac.uk/study/studyinformation/modules/#quick-links-collapse
https://www.exeter.ac.uk/study/studyinformation/modules/#
https://www.exeter.ac.uk/study/studyinformation/modules/#
https://www.exeter.ac.uk/study/studyinformation/modules/#
https://www.exeter.ac.uk/study/studyinformation/modules/#
https://www.exeter.ac.uk/study/studyinformation/modules/#
https://www.exeter.ac.uk/study/studyinformation/modules/#
https://www.exeter.ac.uk/study/studyinformation/modules/#
https://www.exeter.ac.uk/study/studyinformation/modules/#
https://www.exeter.ac.uk/study/studyinformation/modules/#
https://www.exeter.ac.uk/study/studyinformation/modules/#
https://www.exeter.ac.uk/study/studyinformation/modules/#
https://www.exeter.ac.uk/study/studyinformation/modules/#
http

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse, parse_qs
import time
import csv
import re

# === Define SDG keywords ===
sdg_keywords = {
    "SDG 1": ["poverty", "economic justice", "welfare", "deprivation", "social safety nets"],
    "SDG 2": ["food justice", "nutrition policy", "sustainable agriculture", "food insecurity"],
    "SDG 3": ["public health", "wellbeing", "healthcare ethics", "trauma", "mental health"],
    "SDG 4": ["pedagogy", "access to education", "inclusive learning", "literacy", "critical thinking"],
    "SDG 5": ["feminism", "gender identity", "patriarchy", "lgbtq+", "intersectionality"],
    "SDG 6": ["water scarcity", "sanitation ethics", "public hygiene", "infrastructure justice"],
    "SDG 7": ["energy policy", "environmental ethics", "renewables in society"],
    "SDG 8": ["fair wages", "labor rights", "postcolonial economies", "informal labour"],
    "SDG 9": ["innovation policy", "development studies", "smart cities", "digital divides"],
    "SDG 10": ["taxation", "race", "social class", "inequality", "structural discrimination"],
    "SDG 11": ["urban justice", "housing", "mobility", "community resilience"],
    "SDG 12": ["consumerism", "material culture", "waste", "circular economy"],
    "SDG 13": ["climate justice", "global warming ", "Carbon", "ecological crisis", "policy responses"],
    "SDG 14": ["marine policy", "ocean plastic", "fisheries ethics", "blue economy"],
    "SDG 15": ["biodiversity", "deforestation", "indigenous rights", "land justice"],
    "SDG 16": ["conflict", "peacebuilding", "corruption", "democracy", "law", "human rights"],
    "SDG 17": ["international development", "global cooperation", "ngos", "Sustainable Development", "diplomacy"]
}

def extract_module_code(url):
    try:
        parsed_url = urlparse(url)
        query_params = parse_qs(parsed_url.query)
        return query_params.get('moduleCode', [None])[0]
    except:
        return None

def extract_clean_visible_text(driver):
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        script = """
        function getVisibleText(element) {
            if (!element) return '';
            const style = window.getComputedStyle(element);
            if (style.display === 'none' || style.visibility === 'hidden' || style.opacity === '0') return '';
            if (['SCRIPT', 'STYLE', 'NOSCRIPT'].includes(element.tagName)) return '';
            let text = '';
            if (element.nodeType === Node.TEXT_NODE) text = element.textContent.trim();
            for (const child of element.childNodes) {
                text += ' ' + getVisibleText(child);
            }
            return text.trim();
        }
        return getVisibleText(document.body);
        """
        return driver.execute_script(script).strip().lower()
    except:
        try:
            return driver.find_element(By.TAG_NAME, "body").text.strip().lower()
        except:
            return ""

def find_verified_keyword_matches(text, sdg_keywords):
    matched_sdgs = {}
    for sdg, keywords in sdg_keywords.items():
        found_keywords = []
        for kw in keywords:
            if not kw:
                continue
            pattern = rf"\b{re.escape(kw)}\b" if len(kw.split()) == 1 else r"\b" + re.escape(kw) + r"\b"
            if re.findall(pattern, text):
                found_keywords.append(kw)
        if found_keywords:
            matched_sdgs[sdg] = found_keywords
    return matched_sdgs

def main():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-extensions")
    options.page_load_strategy = 'eager'

    try:
        driver = webdriver.Chrome(options=options)
        driver.set_page_load_timeout(30)

        urls = c  # your list of module URLs
        results = []

        for url in urls:
            try:
                driver.get(url)
                time.sleep(3)
                module_code = extract_module_code(url)
                visible_text = extract_clean_visible_text(driver)
                matched_sdgs = find_verified_keyword_matches(visible_text, sdg_keywords)

                if matched_sdgs:
                    matched_str = "; ".join(f"{sdg}: {', '.join(words)}" for sdg, words in matched_sdgs.items())
                    print(url)  # Only this will print
                else:
                    matched_str = "No SDG keywords found"

                results.append([module_code, url, matched_str])

            except Exception as e:
                results.append(["ERROR", url, f"Error: {str(e)}"])

        output_path = filepath
        with open(output_path, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["Module Code", "URL", "Matched SDG Keywords"])
            writer.writerows(results)

    finally:
        if 'driver' in locals():
            driver.quit()

if __name__ == "__main__":
    main()


https://www.exeter.ac.uk/study/studyinformation/modules/?prog=hass&year=2025/6
https://www.exeter.ac.uk/study/studyinformation/modules/info?moduleCode=CLA1001&ay=2025/6&sys=0
https://www.exeter.ac.uk/study/studyinformation/modules/info?moduleCode=CLA1032&ay=2025/6&sys=0
https://www.exeter.ac.uk/study/studyinformation/modules/info?moduleCode=CLA1034&ay=2025/6&sys=0
https://www.exeter.ac.uk/study/studyinformation/modules/info?moduleCode=CLA1406&ay=2025/6&sys=0
https://www.exeter.ac.uk/study/studyinformation/modules/info?moduleCode=CLA1507&ay=2025/6&sys=0
https://www.exeter.ac.uk/study/studyinformation/modules/info?moduleCode=CLA1514&ay=2025/6&sys=0
https://www.exeter.ac.uk/study/studyinformation/modules/info?moduleCode=THE1101&ay=2025/6&sys=0
https://www.exeter.ac.uk/study/studyinformation/modules/info?moduleCode=THE1121&ay=2025/6&sys=0
https://www.exeter.ac.uk/study/studyinformation/modules/info?moduleCode=THE1123&ay=2025/6&sys=0
https://www.exeter.ac.uk/study/studyinformation/modules/i